In [1]:
# ! pip install deepctr==0.8.5 --no-deps
# ! pip install torch==1.7.0 torchvision==0.8.1 
# ! pip install tensorflow-gpu==1.13.1
# ! pip install numba

In [2]:
import sys
sys.path.append('..')
sys.path.append('../../config/')
from config_prosper import *
import os
import pandas as pd
import numpy as np
import tensorflow as tf

from time import time
from deepctr.feature_column import SparseFeat, DenseFeat, get_feature_names,VarLenSparseFeat
from mytools.utils.myfile import savePkl,loadPkl
from mmoe_tf import MMOE
from evaluation import evaluate_deepctr
from tensorflow.python.keras.utils import multi_gpu_model
from tqdm import tqdm as tqdm

BASE_DIR(目录): /home/tione/notebook


In [3]:
# GPU相关设置
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"
# 设置GPU按需增长
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

In [4]:
def loadFeedinfo():
    feed = loadPkl(FEED_INFO_DEAL)
    feed[["bgm_song_id", "bgm_singer_id"]] += 1  # 0 用于填未知
    feed[["bgm_song_id", "bgm_singer_id", "videoplayseconds"]] = \
        feed[["bgm_song_id", "bgm_singer_id", "videoplayseconds"]].fillna(0)
    feed['bgm_song_id'] = feed['bgm_song_id'].astype('int64')
    feed['bgm_singer_id'] = feed['bgm_singer_id'].astype('int64')
    print('feedinfo loading over...')
    return feed

def getFeedembeddings(df):
    #feedembeddings 降维

    feed_embedding_path = os.path.join(FEATURE_PATH,'feedembedings.pkl')
    feed_embeddings = loadPkl(feed_embedding_path)
    df = df.merge(feed_embeddings,on='feedid',how='left')
    dense = [x for x in list(feed_embeddings.columns) if x != 'feedid' ]
    
    return df,dense

def getSvdembeddings(df):
    dense = []
    #userid-feedid svd
    svd_embedding = loadPkl(os.path.join(FEATURE_PATH,'svd_userid_feedid_embedding.pkl'))
    df = df.merge(svd_embedding,on = ['userid'],how='left')
    dense += [x for x in list(svd_embedding.columns) if x not in ['userid']]
                            
    #userid_authorid svd
    svd_embedding = loadPkl(os.path.join(FEATURE_PATH,'svd_userid_authorid_embedding.pkl'))
    df  = df.merge(svd_embedding,on = ['userid'],how='left')
    dense += [x for x in list(svd_embedding.columns) if x not in ['userid']]
    
    #text svd
    svd_embedding = loadPkl(os.path.join(FEATURE_PATH,'texts_svd_embedding.pkl'))
    svd_embedding['feedid']  = svd_embedding['feedid'].astype(np.int32) 
    df  = df.merge(svd_embedding,on = ['feedid'],how='left')
    dense += [x for x in list(svd_embedding.columns) if x not in ['feedid']]
    
    return df, dense
def myLeftjoin(left,right,on):
    return left.merge(right[right[on].isin(left[on])].set_index(on),how='left',left_on=on,right_index=True)
def getHistFeatures(df,hist_features):
    dense = [x for x in hist_features.columns if x not in df.columns and  'hist_seq' not in x ]
    varlen = [x for x in hist_features.columns if 'hist_seq' in x]
    df = df.merge(hist_features[hist_features.userid.isin(df.userid.unique())][['userid','feedid','date_','device'] + dense],how = 'left',on = ['userid','feedid','date_','device'])
    return (df,dense)

In [5]:
class myDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, data: pd.DataFrame,feedinfo,dnn_feature_columns,batch_size=2048, shuffle=True):
        self.data = data.copy()
        self.target = ACTION_LIST
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indexes = np.arange(self.data.shape[0])
        
        self.feedinfo = feedinfo
        self.feed_embeddings = loadPkl(os.path.join(FEATURE_PATH,'feedembedings.pkl'))
#         self.user_feed_svd_embedding = loadPkl(os.path.join(FEATURE_PATH,'svd_userid_feedid_embedding.pkl'))
#         self.user_author_svd_embedding = loadPkl(os.path.join(FEATURE_PATH,'svd_userid_authorid_embedding.pkl'))
#         self.text_svd_embedding = loadPkl(os.path.join(FEATURE_PATH,'texts_svd_embedding.pkl'))
#         self.text_svd_embedding['feedid'] = self.text_svd_embedding['feedid'].astype(int)
        
        self.dnn_feature_columns = dnn_feature_columns
        self.feature_names = get_feature_names(self.dnn_feature_columns)
        
        if self.shuffle:
            print('shuffle data index ing...')
            np.random.shuffle(self.indexes)

    def __len__(self):

        return (self.data.shape[0] // self.batch_size) + 1

    def __getitem__(self, index):
        batch_indexs = self.indexes[index * self.batch_size:(index + 1) *
                                    self.batch_size]
        batch_data = self.data.iloc[batch_indexs, :]
        
        return self.get_feature_on_batch(batch_data)

    def on_epoch_end(self):
        if self.shuffle:
            print('shuffle data index ing...')
            np.random.shuffle(self.indexes)
    def on_epoch_begain(self):
        if self.shuffle:
            print('shuffle data index ing...')
            np.random.shuffle(self.indexes)

    def get_feature_on_batch(self, batch):
        
#         batch = batch.merge(self.user_feed_svd_embedding,on='userid',how='left')
#         batch = batch.merge(self.user_author_svd_embedding,on='userid',how='left')
#         batch = batch.merge(self.text_svd_embedding,on='feedid',how='left')
#         batch = batch.merge(self.feed_embeddings,on='feedid',how='left')
        
        x = {name: batch[name].values for name in self.feature_names}
        for col in ['manual_tag_list','manual_keyword_list','machine_keyword_list']:
            x[col] = np.array(batch[col].tolist())
        y = [batch[y].values for y in ACTION_LIST]
        
        return x,y

In [6]:
DEBUG = False
data = loadPkl(USER_ACTION)
data = data.head(1000000) if DEBUG else data
feedinfo = loadFeedinfo()
# feed_embeddings = loadPkl(os.path.join(FEATURE_PATH,'feedembedings.pkl'))

# user_feed_svd_embedding = loadPkl(os.path.join(FEATURE_PATH,'svd_userid_feedid_embedding.pkl'))
# user_author_svd_embedding = loadPkl(os.path.join(FEATURE_PATH,'svd_userid_authorid_embedding.pkl'))
# text_svd_embedding = loadPkl(os.path.join(FEATURE_PATH,'texts_svd_embedding.pkl'))
embedding_dim = 8
sparse_features = ['userid', 'feedid', 'authorid', 'bgm_song_id', 'bgm_singer_id' ]
dense_features = ['videoplayseconds',]
# dense_features += [x for x in list(user_feed_svd_embedding.columns) if x not in ['userid']]
# dense_features += [x for x in list(user_author_svd_embedding.columns) if x not in ['userid']]
# dense_features += [x for x in list(text_svd_embedding.columns) if x not in ['feedid']]
# dense_features += [x for x in list(feed_embeddings.columns) if x not in ['feedid']]
data = data.merge(feedinfo[[
    'feedid', 'authorid', 'videoplayseconds', 'bgm_song_id',
    'bgm_singer_id'
] + ['manual_tag_list', 'manual_keyword_list', 'machine_keyword_list'
     ]],
                    how='left',
                    on='feedid')

#dense 特征处理
data['videoplayseconds'] = data['videoplayseconds'].fillna(0, )
data['videoplayseconds'] = np.log(data['videoplayseconds'] + 1.0)
train = data[data.date_ != 14]
val = data[data.date_==14]

feedinfo loading over...


In [7]:

fixlen_feature_columns = [
    SparseFeat(feat,
               vocabulary_size = feedinfo[feat].max() + 1,
               embedding_dim=embedding_dim) for feat in sparse_features if feat !='userid'
] + [DenseFeat(feat, 1) for feat in dense_features
] + [SparseFeat('userid',
               vocabulary_size= data['userid'].max() + 1,
               embedding_dim=embedding_dim)]
tag_columns = [
    VarLenSparseFeat(SparseFeat('manual_tag_list',
                                vocabulary_size=TAG_MAX,
                                embedding_dim=8),
                     maxlen=4)
]
key_words_columns = [
    VarLenSparseFeat(SparseFeat('manual_keyword_list',
                                vocabulary_size=KEY_WORDS_MAX,
                                embedding_dim=16),
                     maxlen=4),
    VarLenSparseFeat(SparseFeat('machine_keyword_list',
                                vocabulary_size=KEY_WORDS_MAX,
                                embedding_dim=16),
                     maxlen=4),
]
dnn_feature_columns =  fixlen_feature_columns + tag_columns + key_words_columns

In [17]:
num_tasks = len(ACTION_LIST)
train_model = MMOE(dnn_feature_columns, num_tasks=num_tasks,task_types = ['binary' for i in range(num_tasks)],task_names = ACTION_LIST,num_experts=5,tower_dnn_units_lists = [[16,8] for i in range(num_tasks) ])
# train_model.summary()
train_loader = myDataGenerator(train,feedinfo,dnn_feature_columns,batch_size=4096)
val_loader = myDataGenerator(val,feedinfo,dnn_feature_columns,batch_size=4096 * 4,shuffle = False) # shuffle 必须为False
len(train_loader)
train_model = multi_gpu_model(train_model, gpus=2)
optimizer = tf.keras.optimizers.Adagrad(
    lr=0.01, epsilon=1e-07,
)
train_model.compile("adagrad", loss='binary_crossentropy')

shuffle data index ing...


## offline

In [19]:
best_score = -1
early_stop = 1
no_imporove = 0
for epoch in range(5):
    history = train_model.fit(train_loader,
                              epochs=1, verbose=1,workers = 8,use_multiprocessing=True,max_queue_size=100)
    pred_ans = train_model.predict_generator(val_loader)
    pred_ans = np.concatenate(pred_ans,1)
    pred_ans = pd.DataFrame(pred_ans,columns=ACTION_LIST)
    weightauc,uaucs = evaluate_deepctr(val_loader.data[ACTION_LIST],pred_ans,val_loader.data['userid'].values,ACTION_LIST)
    
    if best_score < weightauc:
        best_score = weightauc
        train_model.save_weights(os.path.join(MODEL_PATH,'tf_models/MMOE_offline'))
        no_imporove = 0    
    else :
        no_imporove += 1
    if no_imporove >= early_stop:
        print('-----stoped on epoch %s ------- ' % (epoch))
        break
    

18013/18013 [==============================] - 645s 36ms/step - loss: 0.2405 - read_comment_loss: 0.0845 - like_loss: 0.0860 - click_avatar_loss: 0.0328 - forward_loss: 0.0175 - comment_loss: 0.0030 - follow_loss: 0.0044 - favorite_loss: 0.0055
【UAUC：0.6727477111586843】 [0.6454365021078887, 0.6341556106579908, 0.7343647988783761, 0.7241241124941185, 0.5851365896601247, 0.7201020009290576, 0.7634151038173138]

Consider using a TensorFlow optimizer from `tf.train`.
18013/18013 [==============================] - 648s 36ms/step - loss: 0.2384 - read_comment_loss: 0.0842 - like_loss: 0.0857 - click_avatar_loss: 0.0325 - forward_loss: 0.0172 - comment_loss: 0.0029 - follow_loss: 0.0043 - favorite_loss: 0.0054
【UAUC：0.6733515209218548】 [0.6459559012303187, 0.6342957596890754, 0.7348110800265281, 0.7245004190225772, 0.5893242076010877, 0.7219778239524416, 0.761434277366449]

Consider using a TensorFlow optimizer from `tf.train`.
18013/18013 [==============================] - 648s 36ms/step - l

In [10]:
train_model.load_weights(os.path.join(MODEL_PATH,'tf_models/MMOE_offline'))

In [11]:
no_imporove , weightauc , best_score

(1, 0.6735051381005702, 0.6736671456723232)

# online

In [20]:
data_loader = myDataGenerator(data,feedinfo,dnn_feature_columns,batch_size=4096)
for epoch in range(1):
    history = train_model.fit(data_loader,
                              epochs=1, verbose=1,workers = 8,use_multiprocessing=True,max_queue_size=100)
    pred_ans = train_model.predict_generator(val_loader)
    pred_ans = np.concatenate(pred_ans,1)
    pred_ans = pd.DataFrame(pred_ans,columns=ACTION_LIST)
    weightauc,uaucs = evaluate_deepctr(val_loader.data[ACTION_LIST],pred_ans,val_loader.data['userid'].values,ACTION_LIST)
train_model.save_weights(os.path.join(MODEL_PATH,'tf_models/MMOE_online'))

shuffle data index ing...
19652/19652 [==============================] - 703s 36ms/step - loss: 0.2352 - read_comment_loss: 0.0835 - like_loss: 0.0851 - click_avatar_loss: 0.0321 - forward_loss: 0.0168 - comment_loss: 0.0028 - follow_loss: 0.0042 - favorite_loss: 0.0053
【UAUC：0.719893921130422】 [0.6955736693185471, 0.6667488665665215, 0.7736452284196123, 0.7829269851432743, 0.661306191569043, 0.779636166792317, 0.8049198973778757]

Consider using a TensorFlow optimizer from `tf.train`.


In [21]:
test = pd.read_csv('../../data/wedata/wechat_algo_data2/test_a.csv')
test = test.merge(feedinfo[['feedid', 'authorid', 'videoplayseconds', 'bgm_song_id', 'bgm_singer_id']+ ['manual_tag_list','manual_keyword_list','machine_keyword_list']], how='left',on='feedid')
test['videoplayseconds'] = test['videoplayseconds'].fillna(0, )
test['videoplayseconds'] = np.log(test['videoplayseconds'] + 1.0)
test[ACTION_LIST] = 0
t1 = time()
test_loader = myDataGenerator(test,feedinfo,dnn_feature_columns,shuffle=False,batch_size=4096*20)
pred_ans = train_model.predict(test_loader)
t2 = time()
print('7个目标行为%d条样本预测耗时（毫秒）：%.3f' % (len(test), (t2 - t1) * 1000.0))
ts = (t2 - t1) * 1000.0 / len(test) * 2000.0
print('7个目标行为2000条样本平均预测耗时（毫秒）：%.3f' % ts)

# 5.生成提交文件
for i, action in enumerate(ACTION_LIST):
    test[action] = pred_ans[i]
test[['userid', 'feedid'] + ACTION_LIST].to_csv(os.path.join(SUMIT_DIR,'tf_mmoe_base4.csv'), index=None, float_format='%.6f')
print('to_csv ok')

7个目标行为4252097条样本预测耗时（毫秒）：9195.646
7个目标行为2000条样本平均预测耗时（毫秒）：4.325
to_csv ok


In [14]:
# 5.生成提交文件
for i, action in enumerate(ACTION_LIST):
    test[action] = pred_ans[i]
test[['userid', 'feedid'] + ACTION_LIST].to_csv(os.path.join(SUMIT_DIR,'tf_mmoe_base3.csv'), index=None, float_format='%.6f')
print('to_csv ok')

to_csv ok


In [22]:
test[['userid', 'feedid'] + ACTION_LIST]

,userid,feedid,read_comment,like,click_avatar,forward,comment,follow,favorite
0,175282,50458,0.030012,0.008492,0.000873,0.036845,0.000375,8.082390e-05,4.684925e-04
1,80036,42329,0.006092,0.004939,0.013913,0.002046,0.000276,1.464099e-03,2.107024e-05
2,145791,85242,0.000164,0.009282,0.000395,0.000137,0.000025,1.215935e-05,4.172325e-07
3,28430,9425,0.000809,0.008853,0.135481,0.029642,0.000847,5.609810e-03,2.330542e-05
4,44393,11866,0.000016,0.000937,0.000040,0.000569,0.000001,2.980232e-07,0.000000e+00
...,...,...,...,...,...,...,...,...,...
4252092,153322,51633,0.000087,0.004937,0.005133,0.007922,0.000044,1.962185e-04,1.725554e-05
4252093,39430,20147,0.004747,0.009092,0.000132,0.000013,0.000023,2.771616e-06,2.384186e-07
4252094,2524,89043,0.000378,0.015480,0.002420,0.000767,0.000068,1.991391e-04,1.257658e-05
4252095,69629,27238,0.023340,0.006304,0.007330,0.000498,0.000580,2.918154e-03,9.969473e-04
